In [1]:
# 예제 5-57 라이브러리 임포트
import pandas as pd
import folium
import json

In [2]:
# 예제 5-58 서울시 시군구별 통계 데이터 불러오기
seoul_sgg_stat = pd.read_excel('./files/4_5_seoul_sgg_stat.xlsx')
seoul_sgg_stat.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민등록인구,종사자수,사업체수
0,11320,도봉구,37.665861,127.031767,2,345041,68669,18455
1,11380,은평구,37.617612,126.922700,8,490253,87693,24179
2,11230,동대문구,37.583801,127.050700,8,364962,143858,32994
3,11590,동작구,37.496504,126.944307,9,406715,103915,19609
4,11545,금천구,37.460097,126.900155,10,253344,223058,30080


In [3]:
# 예제 5-59 서울시 시군구 행정 경계 지도 파일 불러오기
sgg_geojson_file_path = './maps/seoul_sgg.geojson'
seoul_sgg_geo = json.load(open(sgg_geojson_file_path, encoding='utf-8'))
seoul_sgg_geo['features'][0]['properties']

{'SIG_CD': '11320',
 'SIG_KOR_NM': '도봉구',
 'SIG_ENG_NM': 'Dobong-gu',
 'ESRI_PK': 0,
 'SHAPE_AREA': 0.00210990544544,
 'SHAPE_LEN': 0.239901251347}

In [23]:
# 예제 5-60 folium 지도 생성
starbucks_bubble = folium.Map(
    location=[37.573050, 126.979189], tiles ='CartoDB positron' , zoom_start=11 )
#     tiles='CartoDB dark_matter',
#     tiles='CartoDB',
    
# )

In [24]:
starbucks_bubble

In [5]:
# 예제 5-61 서울시 시군구 경계 지도 그리기
def style_function(feature):
    return {
        'opacity': 0.7,
        'weight': 1,
        'color': 'white',
        'fillOpacity':0,
        'dashArray': '5, 5',
    }

folium.GeoJson(
    seoul_sgg_geo,
    style_function=style_function
).add_to(starbucks_bubble)

starbucks_bubble

In [6]:
# 예제 5-62 서울시 시군구별 스타벅스 평균 매장 수 계산
starbucks_mean = seoul_sgg_stat['스타벅스_매장수'].mean()
print(starbucks_mean)

20.0


In [27]:
# 예제 5-63 서울시 시군구별 스타벅스 매장 수를 버블 지도로 시각화
for idx in seoul_sgg_stat.index:
    lat = seoul_sgg_stat.loc[idx, '위도']
    lng = seoul_sgg_stat.loc[idx, '경도']
    count = seoul_sgg_stat.loc[idx, '스타벅스_매장수']

    if count > starbucks_mean:
        fillColor = 'red'
    else:
        fillColor = 'blue'
    
    folium.CircleMarker(
        location=[lat, lng], 
        color='#FFFF00',
        fill_color=fillColor, 
        fill_opacity=0.7,
        weight=1.5,
        radius=count/2
    ).add_to(starbucks_bubble)

starbucks_bubble

In [8]:
# 예제 5-64 서울시 시군구별 스타벅스 매장 수를 단계구분도로 시각화
sgg_geojson_file_path = './maps/seoul_sgg.geojson'
seoul_sgg_geo_2 = json.load(open(sgg_geojson_file_path, encoding='utf-8'))
starbucks_choropleth = folium.Map(
    location=[37.573050, 126.979189],
    tiles='CartoDB dark_matter',
    zoom_start=11
)

folium.Choropleth(
    geo_data=seoul_sgg_geo_2,
    data=seoul_sgg_stat,
    columns=['시군구명', '스타벅스_매장수'],
    fill_color = 'YlGn',
    fill_opacity=0.7,
    line_opacity=0.5,
    key_on='properties.SIG_KOR_NM'
    ).add_to(starbucks_choropleth)

starbucks_choropleth